In [2]:
import collections
import pandas as pd
import mechanicalsoup

In [50]:
ru_y2_metadata = pd.read_csv('HPRC_RU_Y2_Sequel_Metadata_PacBio_HiFi_Submission_Initial.csv')
ru_y2_metadata['flow_cell'] = [flow_cell.split('.')[0] for flow_cell in ru_y2_metadata['filename'].tolist()]
ru_y2_metadata.shape

(54, 24)

In [51]:
ru_y2_qc = pd.read_csv('RU_Y2_HIFI_post_sra_metadata.csv')
ru_y2_qc.rename(columns={'filename':'sample_ID_filename', 'bam_file':'filename'},inplace=True)
print(ru_y2_qc.shape)


(36, 20)


In [52]:
# 18 files are not present in qc sample: HG04187
ru_y2_metadata[~ru_y2_metadata['flow_cell'].isin(ru_y2_qc['flow_cell'].tolist())].shape 

(18, 24)

In [53]:
# all flow cells are present between qc and metadata
ru_y2_qc = ru_y2_qc[ru_y2_metadata['flow_cell'].isin(ru_y2_metadata['flow_cell'].tolist())]

/data/tmp/ipykernel_1196988/2943073831.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  ru_y2_qc = ru_y2_qc[ru_y2_metadata['flow_cell'].isin(ru_y2_metadata['flow_cell'].tolist())]


In [54]:
ru_y2_df = pd.merge( ru_y2_metadata, ru_y2_qc[['flow_cell','filename']], on='flow_cell', how='inner')
ru_y2_df.drop('filename_x', axis=1, inplace=True)
ru_y2_df.rename(columns={'filename_y':'filename'}, inplace=True)

In [55]:
ru_y2_setup = pd.read_csv('RU_Y2_HIFI.csv', header=None)

In [56]:
ru_y2_setup = ru_y2_setup[ru_y2_setup[0].str.contains('primrose')]
ru_y2_setup['filename'] = [filename.split('/')[-1] for filename in ru_y2_setup[0].tolist()]
ru_y2_setup.rename(columns={0:'s3-submission'},inplace=True)

In [59]:
# update primrose working paths
transfer_list = []
for data in ru_y2_setup['s3-submission'].tolist():
	path = data.split('/746FF75B-3C32-42AF-92BB-290BA92CF89A--RU_Y2_HIFI')[1].split('primrose')
	path.insert(1, 'raw_data/PacBio_HiFi/primrose')
	path = ''.join(path)
	release = 's3://human-pangenomics/working/HPRC' + path
	transfer_list.append(release)
ru_y2_setup['s3-working'] = transfer_list
ru_y2_setup[['s3-submission','s3-working']].to_csv('RU_Y2_HiFi.transfer_ec2.csv',index=False, header=None)


In [11]:
ru_y2_df = pd.merge(ru_y2_df,ru_y2_setup, on='filename', how='inner')

In [13]:
ru_y2_df.to_csv('HPRC_RU_Y2_Sequel_Metadata_PacBio_HiFi_Submission_Final.csv',index=False)

In [2]:
ru_y2_df = pd.read_csv('HPRC_RU_Y2_Sequel_Metadata_PacBio_HiFi_Submission_Final.csv')

In [5]:
ru_y2_df.head()

,filetype,sample_ID,library_ID,library_strategy,library_source,library_selection,library_layout,platform,instrument_model,design_description,...,primrose_version,polymerase_version,seq_plate_chemistry_version,generator_facility,generator_contact,notes,flow_cell,filename,s3-submission,s3-working
0,bam,HG00706,HG00706_PB1,WGS,GENOMIC,size fractionation,single,PACBIO_SMRT,PacBio Sequel IIe,HiFi sequencing of 20kb fractionated gDNA,...,P2,c2,C2,Rockefeller University,vgl-lab@rockefeller.edu,NaN,m64055e_210731_145204,m64055e_210731_145204.5mc.hifi_reads.bam,s3://human-pangenomics/submissions/746FF75B-3C...,s3://human-pangenomics/working/HPRC/HG00706/ra...
1,bam,HG00706,HG00706_PB1,WGS,GENOMIC,size fractionation,single,PACBIO_SMRT,PacBio Sequel IIe,HiFi sequencing of 20kb fractionated gDNA,...,P2,c2,C2,Rockefeller University,vgl-lab@rockefeller.edu,NaN,m64055e_210812_050522,m64055e_210812_050522.5mc.hifi_reads.bam,s3://human-pangenomics/submissions/746FF75B-3C...,s3://human-pangenomics/working/HPRC/HG00706/ra...
2,bam,HG00706,HG00706_PB1,WGS,GENOMIC,size fractionation,single,PACBIO_SMRT,PacBio Sequel IIe,HiFi sequencing of 20kb fractionated gDNA,...,P2,c2,C2,Rockefeller University,vgl-lab@rockefeller.edu,NaN,m64055e_210813_141904,m64055e_210813_141904.5mc.hifi_reads.bam,s3://human-pangenomics/submissions/746FF75B-3C...,s3://human-pangenomics/working/HPRC/HG00706/ra...
3,bam,HG00706,HG00706_PB1,WGS,GENOMIC,size fractionation,single,PACBIO_SMRT,PacBio Sequel IIe,HiFi sequencing of 20kb fractionated gDNA,...,P2,c2,C2,Rockefeller University,vgl-lab@rockefeller.edu,NaN,m64055e_210814_234047,m64055e_210814_234047.5mc.hifi_reads.bam,s3://human-pangenomics/submissions/746FF75B-3C...,s3://human-pangenomics/working/HPRC/HG00706/ra...
4,bam,HG01887,HG01887_PB1,WGS,GENOMIC,size fractionation,single,PACBIO_SMRT,PacBio Sequel IIe,HiFi sequencing of 20kb fractionated gDNA,...,P2,c2,C2,Rockefeller University,vgl-lab@rockefeller.edu,NaN,m54306Ue_210830_185043,m54306Ue_210830_185043.5mc.hifi_reads.bam,s3://human-pangenomics/submissions/746FF75B-3C...,s3://human-pangenomics/working/HPRC/HG01887/ra...


# Create BioSample Ids

In [12]:
# biosample check
hprc_biosample = pd.read_csv('/private/groups/hprc/human-pangenomics/documentation-metadata/HPRC_metadata/data/production/hprc-production-biosample-table.tsv', sep='\t')
hprc_biosample.rename(columns={'Sample':'sample_ID'},inplace=True)

ncbi_biosample_df = pd.read_csv('ncbi-biosample-07.31.24.tsv',sep='\t')
ncbi_biosample_df.rename(columns={'BioSample.name':'sample_ID'},inplace=True)

In [34]:
assert set(ru_y2_df[~ru_y2_df['sample_ID'].isin(ncbi_biosample_df.sample_ID.tolist())].sample_ID.tolist()) == set(ru_y2_df[~ru_y2_df['sample_ID'].isin(hprc_biosample.sample_ID.tolist())].sample_ID.tolist())

In [24]:
# Create a MechanicalSoup browser object
manual_resolve = []
browser = mechanicalsoup.Browser()
sample_collection_dict = {sample : {"tissue": None,
                                    "geo_loc_name": None,
                                    "sex": None
                                   } for sample in ru_y2_df[~ru_y2_df['sample_ID'].isin(hprc_production.sample_ID.tolist())].sample_ID.tolist()}

In [25]:
sample_collection_dict

{'HG01887': {'tissue': None, 'geo_loc_name': None, 'sex': None},
 'HG02656': {'tissue': None, 'geo_loc_name': None, 'sex': None},
 'HG03825': {'tissue': None, 'geo_loc_name': None, 'sex': None}}

In [26]:
for sample in sample_collection_dict.keys():
    
    url = 'https://catalog.coriell.org/0/Sections/Search/Sample_Detail.aspx?Ref='+sample+'&Product=DNA' # +'&product=CC'
    page = browser.get(url)
    
    form = page.soup.select('form')[0]

    try:
        country_span = form.select_one('#lblCountry').text
        cell_type_span = form.select_one('#lblCell_Type').text
        sex_span = form.select_one('#lblGender').text
        sample_collection_dict[sample]['tissue'] = cell_type_span
        sample_collection_dict[sample]['geo_loc_name'] = country_span
        sample_collection_dict[sample]['sex'] = sex_span
    except AttributeError:
        print(sample)

In [43]:
# create biosample identifiers and import
sample_collection_dict['HG01887']['Accession'] = 'SAMN42953248'
sample_collection_dict['HG02656']['Accession'] = 'SAMN42953249'
sample_collection_dict['HG03825']['Accession'] = 'SAMN42953250'


In [44]:
sample_collection_df = pd.DataFrame.from_dict(sample_collection_dict,orient='index')

In [45]:
sample_collection_df

,tissue,geo_loc_name,sex,Accession
HG01887,B-Lymphocyte,BARBADOS,Male,SAMN42953248
HG02656,B-Lymphocyte,PAKISTAN,Male,SAMN42953249
HG03825,B-Lymphocyte,BANGLADESH,Female,SAMN42953250


In [13]:
hprc_production = pd.read_csv('hprc-production-07.31.24.tsv',sep='\t',header=1)

In [15]:
# hprc_production[hprc_production['ChildID'].isin(['HG01626'])]

In [41]:
hprc_production[hprc_production['ChildID'].isin(sample_collection_df.index.tolist())]

,ChildID,N,Sex,Subpopulation,Superpopulation,Production Year,Pellets Status: Production,QC: Karyotyping,QC: OMNI Microarray,Pellets Status: UL Nanopore Production,...,Hi-C data generation,Hi-C data submitted to AnVIL,Strand-seq Center Information,Strand-seq sample received,Strand-seq data generation,Strand-seq data submitted to AnVIL,Kinnex Center Information,Kinnex sample received,Kinnex data generation,Kinnex data submitted to AnVIL
62,HG03825,BD25,female,BEB,SAS,Fail Karyo\nEval HPRC+?,Pellets Banked (12/8/20),Completed (3/8/21),Completed (2/16/21),Pellets Banked (12/8/20),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,HG02656,PK13,male,PJL,SAS,Fail Karyo\nEval HPRC+?,Pellets Banked (12/10/20),Completed (3/1/21),Completed (2/16/21),Pellets Banked (12/10/20),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
64,HG01887,BB04,male,ACB,AFR,Fail Karyo\nEval HPRC+?,Pellets Banked (12/8/20),Completed (3/1/21),Completed (2/16/21),Pellets Banked (12/8/20),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
ru_y2_bio_obj = pd.read_csv('RU_Y2_HIFI_BioSampleObject.tsv',sep='\t')
ru_y2_bio_obj.rename(columns={'Sample Name': 'ChildID'},inplace=True)

In [31]:
pd.merge(ru_y2_bio_obj,hprc_production[['ChildID', 'N', 'Subpopulation', 'Superpopulation', 'Production Year']], on='ChildID')

,Accession,ChildID,SPUID,Organism,Tax ID,Isolate,N,Subpopulation,Superpopulation,Production Year
0,SAMN42953248,HG01887,HG01887,Homo sapiens,9606,HG01887,BB04,ACB,AFR,Fail Karyo\nEval HPRC+?
1,SAMN42953249,HG02656,HG02656,Homo sapiens,9606,HG02656,PK13,PJL,SAS,Fail Karyo\nEval HPRC+?
2,SAMN42953250,HG03825,HG03825,Homo sapiens,9606,HG03825,BD25,BEB,SAS,Fail Karyo\nEval HPRC+?


In [26]:
# update biosample tracking
hprc_biosample

,sample_ID,Accession,familyID,Subpopulation,Superpopulation,Production Year
0,HG01891,SAMN17861236,BB05,ACB,AFR,YR1
1,HG00673,SAMN17861654,SH079,CHS,EAS,YR1
2,HG00438,SAMN17861652,SH012,CHS,EAS,YR1
3,HG01071,SAMN17861657,PR17,PUR,AMR,YR1
4,HG00735,SAMN17861655,PR06,PUR,AMR,YR1
...,...,...,...,...,...,...
247,HG03270,SAMN41021631,NaN,ESN,AFR,YR4
248,NA20503,SAMN41021630,NaN,TSI,EUR,YR4
249,NA21106,SAMN41021629,NaN,GIH,SAS,YR4
250,NA19443,SAMN41021640,NaN,LWK,AFR,YR4


# AWS Release Check

In [66]:
# Check sampleID-flow cell id combo in primrose with top level 
aws_df = pd.read_csv('RU_Y2_HIFI.csv', header=None)

In [67]:
aws_df['flow_cell'] = [flow_cell.split('/')[-1].split('.')[0] for flow_cell in aws_df[0].tolist()]
aws_df['sample_id'] = [flow_cell.split('/')[-2] for flow_cell in aws_df[0].tolist()]

In [69]:
aws_df = aws_df[~aws_df['sample_id'].isin(['746FF75B-3C32-42AF-92BB-290BA92CF89A--RU_Y2_HIFI'])]
aws_df = aws_df[~aws_df[0].str.contains('HG04187')]

In [70]:
aws_primrose = aws_df[aws_df['sample_id'].isin(['primrose'])]
aws_primrose['sample_id'] = [sample.split('/')[-3] for sample in aws_primrose[0].tolist()]

/data/tmp/ipykernel_2282294/1651813280.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aws_primrose['sample_id'] = [sample.split('/')[-3] for sample in aws_primrose[0].tolist()]


In [71]:
aws_df = aws_df[~aws_df['sample_id'].isin(['primrose'])]

In [72]:
len([sample for sample in set(aws_primrose['flow_cell'].tolist()) if sample not in set(aws_df['flow_cell'].tolist())])

0

In [73]:
len([sample for sample in set(aws_df['flow_cell'].tolist()) if sample not in set(aws_primrose['flow_cell'].tolist())])

0

In [74]:
print(len(set(aws_df['flow_cell'].tolist())))
print(len(set(aws_df['flow_cell'].tolist())))

36
36


In [75]:
set(aws_df['flow_cell'].tolist()) == set(aws_df['flow_cell'].tolist())

True

In [76]:
aws_primrose['sampleID-flowcellID'] = aws_primrose['sample_id'] + '_' + aws_primrose['flow_cell']

In [81]:
aws_df.head()

,0,flow_cell,sample_id
0,s3://human-pangenomics/submissions/746FF75B-3C...,m64055e_210731_145204,HG00706
1,s3://human-pangenomics/submissions/746FF75B-3C...,m64055e_210812_050522,HG00706
2,s3://human-pangenomics/submissions/746FF75B-3C...,m64055e_210813_141904,HG00706
3,s3://human-pangenomics/submissions/746FF75B-3C...,m64055e_210814_234047,HG00706
8,s3://human-pangenomics/submissions/746FF75B-3C...,m54306Ue_210830_185043,PacBio_HiFi


In [82]:
aws_df = aws_df[~aws_df['sample_id'].isin(['PacBio_HiFi'])]

In [84]:
aws_df['sampleID-flowcellID'] = aws_df['sample_id'] + '_' + aws_df['flow_cell']

/data/tmp/ipykernel_2282294/123282300.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aws_df['sampleID-flowcellID'] = aws_df['sample_id'] + '_' + aws_df['flow_cell']


In [87]:
len(collections.Counter(aws_primrose['sampleID-flowcellID'].tolist()).keys())

36

In [88]:
len(collections.Counter(aws_df['sampleID-flowcellID'].tolist()))

36

In [96]:
for data in list(collections.Counter(aws_df['sampleID-flowcellID'].tolist()).keys()):
	if data not in list(collections.Counter(aws_primrose['sampleID-flowcellID'].tolist()).keys()):
		print(data)

In [97]:
for data in list(collections.Counter(aws_primrose['sampleID-flowcellID'].tolist()).keys()):
	if data not in list(collections.Counter(aws_df['sampleID-flowcellID'].tolist()).keys()):
		print(data)

In [ ]:
ssds staging release     --deployment default     --submission-id 746FF75B-3C32-42AF-92BB-290BA92CF89A    --transfer-csv RU_Y2_HiFi.transfer_ec2.csv     &> RU_Y2_HiFi.transfer_aws.stdout


# SRA

In [92]:
ru_y2_metadata = pd.read_csv('/private/groups/hprc/human-pangenomics/HPRC_metadata/submissions/RU_Y2_HiFi/HPRC_RU_Y2_Sequel_Metadata_PacBio_HiFi_Submission_Final.csv')

In [93]:
ru_y2_metadata.head()

,filetype,sample_ID,library_ID,library_strategy,library_source,library_selection,library_layout,platform,instrument_model,design_description,...,primrose_version,polymerase_version,seq_plate_chemistry_version,generator_facility,generator_contact,notes,flow_cell,filename,s3-submission,s3-working
0,bam,HG00706,HG00706_PB1,WGS,GENOMIC,size fractionation,single,PACBIO_SMRT,PacBio Sequel IIe,HiFi sequencing of 20kb fractionated gDNA,...,P2,c2,C2,Rockefeller University,vgl-lab@rockefeller.edu,NaN,m64055e_210731_145204,m64055e_210731_145204.5mc.hifi_reads.bam,s3://human-pangenomics/submissions/746FF75B-3C...,s3://human-pangenomics/working/HPRC/HG00706/ra...
1,bam,HG00706,HG00706_PB1,WGS,GENOMIC,size fractionation,single,PACBIO_SMRT,PacBio Sequel IIe,HiFi sequencing of 20kb fractionated gDNA,...,P2,c2,C2,Rockefeller University,vgl-lab@rockefeller.edu,NaN,m64055e_210812_050522,m64055e_210812_050522.5mc.hifi_reads.bam,s3://human-pangenomics/submissions/746FF75B-3C...,s3://human-pangenomics/working/HPRC/HG00706/ra...
2,bam,HG00706,HG00706_PB1,WGS,GENOMIC,size fractionation,single,PACBIO_SMRT,PacBio Sequel IIe,HiFi sequencing of 20kb fractionated gDNA,...,P2,c2,C2,Rockefeller University,vgl-lab@rockefeller.edu,NaN,m64055e_210813_141904,m64055e_210813_141904.5mc.hifi_reads.bam,s3://human-pangenomics/submissions/746FF75B-3C...,s3://human-pangenomics/working/HPRC/HG00706/ra...
3,bam,HG00706,HG00706_PB1,WGS,GENOMIC,size fractionation,single,PACBIO_SMRT,PacBio Sequel IIe,HiFi sequencing of 20kb fractionated gDNA,...,P2,c2,C2,Rockefeller University,vgl-lab@rockefeller.edu,NaN,m64055e_210814_234047,m64055e_210814_234047.5mc.hifi_reads.bam,s3://human-pangenomics/submissions/746FF75B-3C...,s3://human-pangenomics/working/HPRC/HG00706/ra...
4,bam,HG01887,HG01887_PB1,WGS,GENOMIC,size fractionation,single,PACBIO_SMRT,PacBio Sequel IIe,HiFi sequencing of 20kb fractionated gDNA,...,P2,c2,C2,Rockefeller University,vgl-lab@rockefeller.edu,NaN,m54306Ue_210830_185043,m54306Ue_210830_185043.5mc.hifi_reads.bam,s3://human-pangenomics/submissions/746FF75B-3C...,s3://human-pangenomics/working/HPRC/HG01887/ra...


In [103]:
ru_y2_metadata = ru_y2_metadata[['sample_ID', 'filename','library_ID', 'library_strategy','library_source','library_selection', 'library_layout',
				'platform','instrument_model','shear_method', 'size_selection', 'ccs_algorithm', 'polymerase_version',
				'seq_plate_chemistry_version', 'generator_facility','generator_contact','filetype']]
ru_y2_metadata['assembly'] = 'unaligned'

In [104]:
ru_y2_sra = pd.merge(ru_y2_metadata, hprc_biosample[['sample_ID','Accession']], on='sample_ID', how='inner')

In [105]:
ru_y2_sra['title'] = ['PacBio HiFi sequencing of ' + sample_id for sample_id in ru_y2_sra['sample_ID'].tolist()]
ru_y2_sra['design_description'] = 'HiFi sequencing of 20kb fractionated gDNA'

In [106]:
ru_y2_sra.rename(columns={'Accession':'biosample_accession'},inplace=True)

In [107]:
ru_y2_sra['flow_cell'] = [flow_cell.split('.')[0] for flow_cell in ru_y2_sra['filename'].tolist()]
ru_y2_sra['library_ID'] = ru_y2_sra['library_ID'] + '_' + ru_y2_sra['flow_cell']

In [108]:
ru_y2_sra.drop(columns=['sample_ID','flow_cell'],inplace=True)

In [ ]:
ru_y2_sra['instrument_model'] = 'Sequel II'

In [111]:
ru_y2_sra.to_csv('sra_biosample_RU_Y2_metadata.tsv',sep='\t', index=False)